# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096867


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:05<01:18,  2.79s/it]

Rendering models:  10%|█         | 3/30 [00:07<01:07,  2.49s/it]

Rendering models:  13%|█▎        | 4/30 [00:07<00:48,  1.85s/it]

Rendering models:  17%|█▋        | 5/30 [00:08<00:35,  1.40s/it]

...

Rendering models:  90%|█████████ | 27/30 [00:14<00:00,  3.33it/s]

Rendering models:  93%|█████████▎| 28/30 [00:15<00:00,  2.57it/s]

Rendering models:  97%|█████████▋| 29/30 [00:15<00:00,  2.51it/s]

Rendering models: 100%|██████████| 30/30 [00:16<00:00,  3.09it/s]

Alexander00                             0.000282
Jorgearcelarreta                        0.000350
not-logged-in-e25a190c339475300978      0.000412
ChelseaB98                              0.029738
not-logged-in-46a23861c6275e310c67      0.000250
jnarayanbvg                             0.007864
awright5                                0.001423
izzyathogwarts                          0.004013
maxbosetti                              0.015785
not-logged-in-f09819078c043556a4da      0.000152
not-logged-in-22877e3a7af4aafb750a      0.001480
not-logged-in-44c2a2616c2e5fbe7859      0.008956
not-logged-in-92e92166c0411a000b1a      0.025127
Giantjack14                           136.991545
zac8151                                 0.000318
peakscience7                            0.008196
acapirala                               0.000260
pangeli5                                0.000247
not-logged-in-3ae935e3afaee843f530      0.015525
Russell_Hoch                            0.761147
not-logged-in-a018d7

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())